In [ ]:
# Path to the .db file in Google Drive
db_path = '/content/drive/My Drive/ColabNotebooks/Hackathon/articles1.db'

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/ColabNotebooks/Hackathon/'


articles1.db


In [6]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/content/drive/My Drive/ColabNotebooks/Hackathon/articles1.db'
conn = sqlite3.connect(db_path)

# Extract the table and select only 'title' and 'summary'
query = "SELECT title, summary FROM articles"  # Replace `your_table_name` with the actual table name
data = pd.read_sql_query(query, conn)

# Display the extracted data
print(data.head())

# Close the connection
conn.close()

                                               title  \
0  Tutorial on Answering Questions about Images w...   
1  pix2code: Generating Code from a Graphical Use...   
2  Learning Visual Question Answering by Bootstra...   
3  Document Image Coding and Clustering for Scrip...   
4  hyper-sinh: An Accurate and Reliable Function ...   

                                             summary  
0    Together with the development of more accura...  
1    Transforming a graphical user interface scre...  
2    Attention mechanisms in biological perceptio...  
3    The paper introduces a new method for discri...  
4    This paper presents the 'hyper-sinh', a vari...  


In [7]:
# Embed
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for the summaries
data['embeddings'] = data['summary'].apply(lambda x: model.encode(x))

# Verify embeddings
print(data['embeddings'].head())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0    [-0.057094697, -0.075381555, 0.031874906, -0.0...
1    [-0.07215581, -0.028809989, 0.0063608433, -0.0...
2    [0.019481376, -0.07857328, 0.020384995, 0.0164...
3    [-0.053401235, 0.05173978, -0.016036237, -0.04...
4    [-0.03320878, -0.09583652, 0.011146371, 0.0009...
Name: embeddings, dtype: object


In [11]:
# install mylvus
# !pip install pymilvus
!pip install pymilvus sentence-transformers

In [19]:
import os

# Access the Zilliz secret
zilliz_token = os.environ.get("zilliz")
print("Zilliz Token:", zilliz_token)  # Verify that the token is retrieved (optional)


Zilliz Token: None


In [20]:
from pymilvus import connections

# Correct the URI (remove the "uri = " prefix)
connections.connect(
    alias="default",
    uri="https://in03-679d74540007495.serverless.gcp-us-west1.cloud.zilliz.com",  # Correct URI
    token='d4c29d2d2779e53ca6c7bf2021b712808417a983b7bb2d060231f1f6ce9ecb9a45753287a84541659a149e6a94b4cbdd24e67ef0' # Use the secret token
)

# Verify the connection
from pymilvus import utility
print("Connected to Zilliz Cloud. Collections:", utility.list_collections())


Connected to Zilliz Cloud. Collections: []


In [22]:
#3. Prepare Data and Embeddings

import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the SQLite database
db_path = '/content/drive/My Drive/ColabNotebooks/Hackathon/articles1.db'
conn = sqlite3.connect(db_path)
query = "SELECT title, summary FROM articles"  # Replace `your_table_name` with the actual table name
data = pd.read_sql_query(query, conn)
conn.close()

# Generate embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
data['embeddings'] = data['summary'].apply(lambda x: model.encode(x))

# Check the data
print(data.head())


                                               title  \
0  Tutorial on Answering Questions about Images w...   
1  pix2code: Generating Code from a Graphical Use...   
2  Learning Visual Question Answering by Bootstra...   
3  Document Image Coding and Clustering for Scrip...   
4  hyper-sinh: An Accurate and Reliable Function ...   

                                             summary  \
0    Together with the development of more accura...   
1    Transforming a graphical user interface scre...   
2    Attention mechanisms in biological perceptio...   
3    The paper introduces a new method for discri...   
4    This paper presents the 'hyper-sinh', a vari...   

                                          embeddings  
0  [-0.057094697, -0.075381555, 0.031874906, -0.0...  
1  [-0.07215581, -0.028809989, 0.0063608433, -0.0...  
2  [0.019481376, -0.07857328, 0.020384995, 0.0164...  
3  [-0.053401235, 0.05173978, -0.016036237, -0.04...  
4  [-0.03320878, -0.09583652, 0.011146371, 0.0009..

In [23]:
#4. Define Schema and Create a Collection

from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Define the schema
dim = len(data['embeddings'].iloc[0])
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
schema = CollectionSchema(fields, description="Schema for article embeddings")

# Create collection
collection_name = "articles"
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

collection = Collection(name=collection_name, schema=schema)
print(f"Collection {collection_name} created.")


Collection articles created.


In [29]:
print(collection.schema)


{'auto_id': True, 'description': 'Schema for article embeddings', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 500}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}


In [30]:
#5. Insert Data into Zilliz Cloud

# Prepare data for insertion
entities = [
    data['title'].tolist(),                # Titles (up to 500 characters)
    data['embeddings'].tolist()            # 384-dimensional embeddings
]

# Insert data into the collection
collection.insert(entities)
print(f"Inserted {len(data)} records into the collection.")

Inserted 36000 records into the collection.


In [31]:
# 6. Build an Index
# Build an index on the embedding field
index_params = {
    "index_type": "IVF_FLAT",  # Use IVF_FLAT for simplicity; you can use other index types like IVF_SQ8 or HNSW
    "metric_type": "COSINE",  # Metric for similarity search
    "params": {"nlist": 128}
}

collection.create_index(field_name="embedding", index_params=index_params)
print(f"Index created for collection {collection_name}.")


Index created for collection articles.


In [33]:
# Load the collection
collection.load()

# Search in the collection
results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    output_fields=["title"]
)

# Display search results
print("Top results:")
for hit in results[0]:
    print(f"Title: {hit.entity.get('title')}, Score: {hit.score}")


Top results:
Title: Managing extreme AI risks amid rapid progress, Score: 0.6336081624031067
Title: Safetywashing: Do AI Safety Benchmarks Actually Measure Safety Progress?, Score: 0.6235601902008057
Title: Evolutionary Computation and AI Safety: Research Problems Impeding
  Routine and Safe Real-world Application of Evolution, Score: 0.6022111773490906


In [40]:
# 7. Query the Collection
# User query
user_query = "What are the top 3 topics in AI safety?"
query_embedding = model.encode(user_query)

# Search in the collection
results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    output_fields=["title"]
)

# Display search results
print("Top results:")
for hit in results[0]:
    print(f"Title: {hit.entity.get('title')}, Score: {hit.score}")


Top results:
Title: Managing extreme AI risks amid rapid progress, Score: 0.6337218284606934
Title: Safetywashing: Do AI Safety Benchmarks Actually Measure Safety Progress?, Score: 0.6236190795898438
Title: Evolutionary Computation and AI Safety: Research Problems Impeding
  Routine and Safe Real-world Application of Evolution, Score: 0.6020842790603638
